In [ ]:
import pandas as pd

In [ ]:
# Load csv files into DataFrames
sf_78620_df = pd.read_csv("/content/78620_SF.csv")
sf_78642_df = pd.read_csv("/content/78642_SF.csv")
sf_78660_df = pd.read_csv("/content/78660_SF.csv")
sf_78664_df = pd.read_csv("/content/78664_SF.csv")
sf_78666_df = pd.read_csv("/content/78666_SF.csv")
c_78701_df = pd.read_csv("/content/78701_CONDO.csv")
sf_78744_df = pd.read_csv("/content/78744_SF.csv")
c_78746_df = pd.read_csv("/content/78746_CONDO.csv")
sf_78746_df = pd.read_csv("/content/78746_SF.csv")
c_78758_df = pd.read_csv("/content/78758_CONDO.csv")

# Review sample DataFrame
sf_78620_df.head()

,#,Listing ID,St,MLS Area,Address,# Beds,# Full Baths,# Half Baths,# Garage Spaces,Levels,Year Built,Acres,SqFt,List Price,LP$/SqFt,Close$/SqFt,Close Price,Close Date,DOM,CDOM
0,1,2819373,C,BL,235 Mystic Overlook,2,2,0,0.0,1,2018,2.50,"1,984","$650,000",$327.62,$300.00,"$595,200",02/09/2024,118,108
1,1,4751347,C,BL,901 Beauchamp Rd,3,2,0,2.0,1,2013,1.50,"1,827","$689,000",$377.12,$369.46,"$675,000",11/08/2023,64,64
2,1,3471099,C,BL,1239 Whitetail Dr,3,2,0,0.0,2,1985,1.23,"1,457","$875,000",$600.55,$519.56,"$757,000",01/24/2024,33,31
3,1,9152329,C,BL,447 Miss Donna Ln,4,2,1,0.0,1,2005,4.64,"2,535","$800,000",$315.58,$309.66,"$785,000",02/23/2024,15,15
4,1,5992438,C,BL,TBA Mystic Overlook Rd,3,2,0,2.0,1,1985,20.00,"2,847","$1,389,000",$487.88,$421.95,"$1,201,305",04/09/2024,51,141


In [ ]:
# Create list of DataFrames
all_dfs = [sf_78620_df, sf_78642_df, sf_78660_df, sf_78664_df, sf_78666_df, c_78701_df, sf_78744_df, c_78746_df, sf_78746_df, c_78758_df]

# Define a list of names corresponding to the dataframes
df_names = ["sf_78620_df", "sf_78642_df", "sf_78660_df", "sf_78664_df", "sf_78666_df", "c_78701_df", "sf_78744_df", "c_78746_df", "sf_78746_df", "c_78758_df"]

In [ ]:
# Build function for basic cleaning of all DataFrames in list
def clean_dataframe(df, name):

    # Remove unnecessary columns
    df.drop(columns=["#", "Listing ID", "St", "MLS Area", "Address", "Close Date", "DOM", "List Price", "LP$/SqFt", "Close$/SqFt"], inplace=True)

    # Extract the zip code from the dataframe name
    zip_code_parts = name.split("_")
    if len(zip_code_parts) == 3:  # Check if the name has the expected format
        zip_code = zip_code_parts[1]  # Extract the zip code from the name
        df["Zip Code"] = zip_code
    else:
        zip_code = ""
        df["Zip Code"] = zip_code

    # Extract the home type from the dataframe name
    home_type = "Single Family" if name.startswith("sf") else "Condo"

    # Add a "Home Type" column with the extracted home type value to the dataframe
    df["Home Type"] = home_type

    # Remove non-numeric characters from columns with dollar signs
    columns_with_dollar_signs = ["Close Price"]
    for column in columns_with_dollar_signs:
        df[column] = df[column].replace("[\$,]", "", regex=True).astype(float)

    # Combine full and half bath columns to create a new column for total bathrooms
    df["Total Baths"] = df["# Full Baths"] + 0.5 * df["# Half Baths"]

    # Drop original full and half bath columns
    df.drop(columns=["# Full Baths", "# Half Baths"], inplace=True)

    # Remove commas from values in the "SqFt" column
    df["SqFt"] = df["SqFt"].str.replace(",", "")

    # Convert "SqFt" and "Levels" columns to numeric
    df[["SqFt", "Levels"]] = df[["SqFt", "Levels"]].apply(pd.to_numeric, errors="coerce")

    return df

# Loop through each dataframe in the list and apply cleaning function
for i, df in enumerate(all_dfs):
    name = df_names[i]  # Get the name corresponding to the current dataframe
    all_dfs[i] = clean_dataframe(df, name)

# Review sample DataFrame after cleaning
sf_78620_df.head()

,# Beds,# Garage Spaces,Levels,Year Built,Acres,SqFt,Close Price,CDOM,Zip Code,Home Type,Total Baths
0,2,0.0,1.0,2018,2.50,1984,595200.0,108,78620,Single Family,2.0
1,3,2.0,1.0,2013,1.50,1827,675000.0,64,78620,Single Family,2.0
2,3,0.0,2.0,1985,1.23,1457,757000.0,31,78620,Single Family,2.0
3,4,0.0,1.0,2005,4.64,2535,785000.0,15,78620,Single Family,2.5
4,3,2.0,1.0,1985,20.00,2847,1201305.0,141,78620,Single Family,2.0


In [ ]:
# Combine all cleaned dataframes into one dataframe
combined_df = pd.concat(all_dfs)

# Reset index to ensure continuous index values
combined_df.reset_index(drop=True, inplace=True)

# Review the combined dataframe
print(combined_df.head())

   # Beds  # Garage Spaces  Levels  Year Built  Acres  SqFt  Close Price  \
0       2              0.0     1.0        2018   2.50  1984     595200.0   
1       3              2.0     1.0        2013   1.50  1827     675000.0   
2       3              0.0     2.0        1985   1.23  1457     757000.0   
3       4              0.0     1.0        2005   4.64  2535     785000.0   
4       3              2.0     1.0        1985  20.00  2847    1201305.0   

   CDOM Zip Code      Home Type  Total Baths  
0   108    78620  Single Family          2.0  
1    64    78620  Single Family          2.0  
2    31    78620  Single Family          2.0  
3    15    78620  Single Family          2.5  
4   141    78620  Single Family          2.0  


In [ ]:
combined_df.dtypes

# Beds               int64
# Garage Spaces    float64
Levels             float64
Year Built           int64
Acres              float64
SqFt                 int64
Close Price        float64
CDOM                 int64
Zip Code            object
Home Type           object
Total Baths        float64
dtype: object

In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.2 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from category_encoders import TargetEncoder

# Combine "Zip Code" and "Home Type" (categorical features) into a single dataframe
encoder_df = combined_df[["Zip Code", "Home Type"]]

# Perform target encoding for categorical features
encoder = TargetEncoder()
encoded_features = encoder.fit_transform(encoder_df, combined_df["Close Price"])

# Prepare continuous features
continuous_features = ["# Beds",
                       "# Garage Spaces",
                       "Levels",
                       "Year Built",
                       "Acres",
                       "SqFt",
                       "CDOM",
                       "Total Baths"]
boost_train_continuous = combined_df[continuous_features]

# Combine target encoded categorical features and continuous features
x_train_boost_encoded = pd.concat([boost_train_continuous, encoded_features], axis=1)

# Split data into training and testing sets
y = combined_df["Close Price"]
x_train, x_test, y_train, y_test = train_test_split(x_train_boost_encoded, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Parameter tuning
# Define the parameter grid for tuning
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.2, 0.3],
    'min_child_weight': [1, 2, 3],
    'gamma': [0.1, 0.2, 0.3],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0.1, 0.5, 1],
    'reg_lambda': [0.1, 0.5, 1],
    'n_estimators': [100, 200, 300]
}

# Initialize XGBoost regressor
xgb_model = xgb.XGBRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1, verbose=3)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

# Print the best parameters found
print("Best parameters found by grid search:")
print(grid_search.best_params_)

# Get the best estimator
best_xgb_model = grid_search.best_estimator_

In [ ]:
'''# Build the XGBRegressor model
# Trial 1 using params found in test_data
xgb_model_1 = xgb.XGBRegressor(colsample_bytree=0.9,
                             gamma=0.1,
                             learning_rate=0.1,
                             max_depth=3,
                             min_child_weight=3,
                             n_estimators=300,
                             reg_alpha=0.1,
                             reg_lambda=0.1,
                             subsample=0.7)

# Fit the model to the training data
xgb_model_1.fit(x_train, y_train)

# Make predictions
predictions_1 = xgb_model_1.predict(x_test)'''

In [ ]:
'''from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error, mean_squared_log_error, explained_variance_score
import numpy as np

# Evaluate the model, model trial 1
# Calculate R-squared
r_squared = r2_score(y_test, predictions_1)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions_1)

# Calculate Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, predictions_1, squared=False)

# Calculate Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape = mean_absolute_percentage_error(y_test, predictions_1)

# Calculate Median Absolute Error (MedAE)
medae = median_absolute_error(y_test, predictions_1)

# Calculate Mean Squared Logarithmic Error (MSLE)
msle = mean_squared_log_error(y_test, predictions_1)

# Calculate Explained Variance Score
explained_var_score = explained_variance_score(y_test, predictions_1)

# Print the evaluation metrics
print("R-squared:", r_squared)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)
print("Median Absolute Error (MedAE):", medae)
print("Mean Squared Logarithmic Error (MSLE):", msle)
print("Explained Variance Score:", explained_var_score)'''

R-squared: 0.7114856923757661
Mean Absolute Error (MAE): 106824.91361350575
Root Mean Squared Error (RMSE): 301033.64189912804
Mean Absolute Percentage Error (MAPE): 14.10671194086083
Median Absolute Error (MedAE): 41617.46875
Mean Squared Logarithmic Error (MSLE): 0.03850193101829329
Explained Variance Score: 0.7116164270127023
